In [1]:
data Exp = Num Int
         | Exp :+: Exp
         | Exp :/: Exp
         | Var String
         | Let String Int Exp

type Env a b = a -> Maybe b

type ExpEnv = Env String Int

insert :: Eq a => a -> b -> Env a b -> Env a b
insert k v env k' = if k == k' then Just v else env k'

In [2]:
eval :: Exp -> Maybe Int
eval (Num n)     = Just n
eval (Var i)     = Just 0
eval (e1 :+: e2) = (+) <$> eval e1 <*> eval e2
eval (e1 :/: e2) = eval e2 >>= \e2' -> case e2' of 
    0 -> Nothing
    _ -> div <$> eval e1 <*> Just e2'
eval (Let x n e) = Just 0

In [3]:
eval $ Let "x" 3 $ Var "x" :+: Num 3

Just 0

In [12]:
newtype State s a = ST (s -> (a,s))

-- get liefert immer das s von dem state aber nur im monad kontext do oder bind
get :: State s s
get = ST  $ \s -> (s,s)

put:: s -> State s ()
put ns = ST $ const ((), ns)

runState :: s -> State s a -> a
runState init (ST sf) = fst $ sf init

In [13]:
instance Applicative (State s) where

  pure = return
  
  --(<*>) :: State s (a -> b) -> State s a -> State s b
  ST sf <*> ST sa = ST $ \s ->
     let (f,s1) = sf s
         (x,s2) = sa s1 in
       (f x, s2)
  
instance Functor (State s) where

  fmap f (ST sf) = ST $ \s ->
     let (x,s1) = sf s in
        (f x, s1)
  
instance Monad (State s) where

  -- return :: a -> State s a
  return x = ST (\s -> (x,s))

  -- (>>=) :: State s a -> (a -> State s b) -> State s b
  st_sf >>= f = ST $ \s -> 
     let ST sf = st_sf
         (x, s1) = sf s
         ST sg = f x in
       sg s1

Line 20: Use tuple-section
Found:
\ s -> (x, s)
Why not:
(x,)

In [21]:
evalState :: Exp -> State ExpEnv (Maybe Int)
evalState (Num n)     = pure (Just n)
evalState (Var i)     = do
                        env <- get
                        return $ env i
evalState (e1 :+: e2) = do
                        res1 <- evalState e1
                        res2 <- evalState e2
                        return  $ (+) <$> res1 <*> res2
evalState (e1 :/: e2) = do
                        e' <- evalState e2
                        case e' of
                            Just 0 -> return Nothing
                            _ -> do
                                e1' <- evalState e1
                                return $ div <$> e1' <*> e'

evalState (Let x n e) = do
                        env <- get
                        let env' = insert x n env
                        put env'
                        x <- evalState e
                        put env
                        return x


e1 :: Exp
e1 = (Let "x" 42 (Var "x") :+: Var "x1")
 
e2 :: Exp
e2 = (Let "y" 42 (Var "y") :+: Var "y")
 
e3 :: Exp
e3 = (Let "y" 42 (Var "y") :/: Num 0)


testEnv :: ExpEnv
testEnv = insert "x1" 1 (insert "y1" 2 (const Nothing))

runState testEnv $ evalState e1
runState testEnv $ evalState e2
runState testEnv $ evalState e3


Line 28: Redundant bracket
Found:
(Let "x" 42 (Var "x") :+: Var "x1")
Why not:
Let "x" 42 (Var "x") :+: Var "x1"Line 31: Redundant bracket
Found:
(Let "y" 42 (Var "y") :+: Var "y")
Why not:
Let "y" 42 (Var "y") :+: Var "y"Line 34: Redundant bracket
Found:
(Let "y" 42 (Var "y") :/: Num 0)
Why not:
Let "y" 42 (Var "y") :/: Num 0

Just 43

Nothing

Nothing

In [ ]:
{-# LANGUAGE LambdaCase #-}

evalState :: Exp -> State ExpEnv (Maybe Int)
evalState (Num n)     = pure (Just n)
evalState (Var i)     = get >>= \env -> return $ env i
evalState (e1 :+: e2) = evalState e1 >>= \res1 -> evalState e2 >>= \res2 -> return  $ (+) <$> res1 <*> res2
evalState (e1 :/: e2) = evalState e2 >>= \case 
                                    Just 0 -> return Nothing
                                    e2' -> evalState e1 >>= \e1' -> return $ div <$> e1' <*> e2'

In [ ]:
main = do
    let exp = Let "x" 42 (Var "x") :+: Var "x"
    let (result, finalState) = runState (evalState exp) $ const Nothing
    putStrLn $ "Ergebnis: " ++ show result

main

Ergebnis: Just 84